# 04 model building

**Objectives:**
* Using the filtered feature subset, train a GBM model that predicts the AB1_StdDev_Yen variable
* Use the model as input for computing partial dependence plots
* Use the model as input for Anchors / LIME explanations